In [76]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import time

import torch
from torchvision import datasets, transforms
from torch.utils.data import Dataset, Subset, DataLoader
from torch import nn, optim
import torch.nn.functional as f

import pandas as pd

from sklearn.model_selection import train_test_split

### Loading leaf data:

In [ ]:
# data_dir = '/...../Plant_leave_diseases_dataset_with_augmentation'

E = 128  # Matrix size

data_transform = transforms.Compose([
    transforms.Resize([E, E]),
    transforms.ToTensor()
])
dataset = datasets.ImageFolder(data_dir, transform=data_transform)

n_classes = len(dataset.classes)
batch_size = 64
workers = 0

# label-indice-conversions
c_to_i = dataset.class_to_idx
i_to_c = dict((v, h) for h, v in c_to_i.items())

# Split the indices in a stratified way.
indices = np.arange(len(dataset))
train_indices, test_indices = train_test_split(indices, test_size=0.09514, stratify=dataset.targets)

### Make train and test datasets, loaders, images and labels

In [71]:
train_dataset = Subset(dataset, train_indices)
test_dataset = Subset(dataset, test_indices)

train_loader = iter(DataLoader(train_dataset, batch_size=batch_size, num_workers=workers, shuffle=False))
test_loader = iter(DataLoader(test_dataset, batch_size=batch_size, num_workers=workers, shuffle=False))

X_train, y_train = train_loader.next()
X_test, y_test = test_loader.next()

### Pytorch implementation of the 9-layer net (geetharamani et.al., 2019):

In [66]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.poolconv1 = nn.Conv2d(32, 16, 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 16, 3)
        self.poolconv2 = nn.Conv2d(16, 8, 1)
        self.conv3 = nn.Conv2d(8, 8, 3)
        self.fc1 = nn.Linear(8 * 28 * 28, 1568)
        self.fc2 = nn.Linear(1568, 128)
        self.fc3 = nn.Linear(128, n_classes)

    def forward(self, x):
        x = f.relu(self.conv1(x))
        x = self.poolconv1(self.pool(x))
        x = f.relu(self.conv2(x))
        x = self.poolconv2(self.pool(x))
        x = f.relu(self.conv3(x))
        x = torch.flatten(x, 1)
        x = f.relu(self.fc1(x))
        x = f.relu(self.fc2(x))
        x = f.relu(self.fc3(x))
        return x

model = Net()

### Instaniate model with training set, reset gradients, set loss fuction, and SET optimizer:

In [40]:
out = model(X_train)
model.zero_grad()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

### Train the model and save to disc:

In [ ]:
t0 = time.time()

epochs = 10

for epoch in range(epochs):

    running_loss = 0.0

    for i, data in enumerate(train_loader, 0):

        inputs, labels = data

        optimizer.zero_grad()

        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        di = 20
        running_loss += loss.item()
        if i % di == di - 1:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / di))
            running_loss = 0.0

print('Finished Training')

t_d0 = time.time() - t0
print('Took {} to train'.format(t_d0))

print('Saving "leaf_0.pth"')
torch.save(model.state_dict(), 'leaf_0.pth')

### Loading and testing the model on the test dataset:

In [67]:
model.load_state_dict(torch.load('leaf_0.pth'))

# "outputs" conatain a list with an "energy"-value for each class. From this list the class with max energy is selected.
outputs = model(X_test)
_, predicted = torch.max(outputs, 1)

# Make a ground truth.
al = []
for i in range(len(y_test)):
    al.append(y_test.numpy()[i])

# Make a table with predicted and add gound truth.
df = pd.DataFrame({'predicted': predicted})
df['ground truth'] = al
df

,predicted,ground truth
0,38,38
1,25,7
2,25,23
3,25,25
4,29,6
...,...,...
59,12,13
60,16,17
61,27,24
62,36,36


### Accuracy for the full test data set:

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
        100 * correct / total))

### Accuracy for each class:

In [ ]:
correct_pred = {classname: 0 for classname in tuple(dataset.classes)}
total_pred = {classname: 0 for classname in tuple(dataset.classes)}

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[dataset.classes[label]] += 1
            total_pred[dataset.classes[label]] += 1

# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("{:.1f}%: {:.0f}, {:5s}".format(accuracy, c_to_i[classname], classname))